In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym

In [ ]:
env = gym.make("MountainCar-v0", domain_randomize=True)

In [ ]:
from torch import nn
import torch.nn.functional as F
from gymnasium import spaces

class TDLambda(nn.Module):
    def __init__(self, lam: float = 0.9, discount: float = 0.95, alpha: float = 0.05, action_size = 3, observation_size = 2):
        self.lam = lam
        self.discount = discount
        self.alpha = alpha

        # environment specific vars
        action_space = spaces.Discrete(action_size)
        self.action_size = action_size
        self.observation_size = observation_size
        